# 통계기반 자연어 처리 

## NLP의 접근 방식 
1. 규칙 기반 
    - 사람의 규칙(문법 / 패턴)을 정의
2. 통계 기반
    - 단어의 빈도, 확률, 통계를 활용
3. 딥러닝 기반 
    - 대규모 데이터 + 신경망 모델 활용

## N-gram 근사 
- Unigram 
    - N = 1 
    - 독립 단어 (하나의 단어)
- Bigram
    - N = 2
    - 바로 앞의 단어를 고려 
- Trigram
    - N = 3
    - 앞의 2개의 단어를 고려 

## 로그 확률
- 단어의 조건부 확률은 매우 작은 값 (0.0001, 0.00001)
- 조건부의 확률들 끼리 곱하게 되면 -> 0에 가까운 값 -> 계산이 불안정 
- 이러한 문제를 해결하기 위해 log 값을 사용 

## 혼란도(Preplexity)
- 문장을 얼마나 햇갈려하는지를 수치로 표현한 값
- 값이 높다면 -> 문장 이해도가 내려간다.
- 값이 낮다면 -> 문장 이해도가 올라간다. 

### BoW 
- 문서의 단어 순서는 무시 
- 각 문서에서 단어가 몇번 등장했는가?(빈도수)
- 가장 기본적인 백터화 방법 

In [1]:
# 샘플 데이터셋 생성 
docs = [
    "영화가 정말 재미있었다", 
    "영화가 너무 지루하다", 
    "배우의 연기가 너무 좋았다."
]

In [2]:
tokens = [doc.split() for doc in docs]
tokens

[['영화가', '정말', '재미있었다'], ['영화가', '너무', '지루하다'], ['배우의', '연기가', '너무', '좋았다.']]

In [3]:
# tokens에서 각각의 단어들을 하나의 리스트로 생성하고 중복은 제거 
# tokens의 2차원 리스트를 1차원으로 변환 
vocab1 = []
for token in tokens:
    # token -> tokens의 각 원소들 -> 1차원 리스트 
    for word in token:
        # word : token이라는 1차원 리스트의 각각의 원소들 
        vocab1.append(word)
vocab1 = list(set(vocab1))

In [4]:
vocab1

['재미있었다', '연기가', '너무', '지루하다', '영화가', '배우의', '좋았다.', '정말']

In [5]:
vocab = list(set(sum(tokens, [])))

In [6]:
# Bow 행렬을 하나 생성 
# tokens의 문장에 vocab의 단어가 몇개 포함되어있는가?
bow_list = []
vocab.sort()
for doc in tokens:
    # row = [ doc.count(word) for word in vocab ]
    row = []
    for word in vocab:
        # count() 함수는 list에서 인자의 값과 같은 데이터가 몇개 있는가?
        row.append(doc.count(word))
    bow_list.append(row)

In [7]:
import pandas as pd 

In [8]:
df_bow = pd.DataFrame(bow_list, columns=vocab)

In [9]:
df_bow

,너무,배우의,연기가,영화가,재미있었다,정말,좋았다.,지루하다
0,0,0,0,1,1,1,0,0
1,1,0,0,1,0,0,0,1
2,1,1,1,0,0,0,1,0


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)

In [12]:
df_vectorizer = pd.DataFrame(X.toarray(), 
    columns = vectorizer.get_feature_names_out())

In [13]:
df_vectorizer

,너무,배우의,연기가,영화가,재미있었다,정말,좋았다,지루하다
0,0,0,0,1,1,1,0,0
1,1,0,0,1,0,0,0,1
2,1,1,1,0,0,0,1,0


### TF-IDF 
- Bow 모델(단순 빈도 모델)의 한계를 보완하기 위한 통계 기반 기법

- TF
    - 문서(문장들의 집합) 내의 빈도가 높을수록 값이 큼
    - 자주 등장 할수록 그 문서에서 중요하다고 판단 
- IDF
    - 문장 내에서의 단어의 희소성

In [14]:
import math 

In [15]:
# 단어사전의 길이 
V = len(vocab)
# 전체 문서의 길이 
N = len(docs)

In [16]:
tokens

[['영화가', '정말', '재미있었다'], ['영화가', '너무', '지루하다'], ['배우의', '연기가', '너무', '좋았다.']]

In [17]:
# 단어의 개수를 생성 
word_cnt = {
    w : sum(1 for doc in tokens if w in doc) for w in vocab
}
word_cnt

{'너무': 2,
 '배우의': 1,
 '연기가': 1,
 '영화가': 2,
 '재미있었다': 1,
 '정말': 1,
 '좋았다.': 1,
 '지루하다': 1}

In [18]:
# TF 계산식 함수 
def tf(word, doc):
    # word : 단어 사전의 각 원소
    # doc : tokens 각 원소 
    # doc.count(word) : 문장에서 특정 단어의 개수 
    # len(doc) : 문장의 단어의 개수
    result = doc.count(word) / len(doc)
    return result

In [19]:
# IDF 계산식 함수 
def idf(word):
    # word : 단어 사전의 각 원소
    # N : docs의 길이 -> 문장들의 개수 
    # word_cnt[word] -> 문서에서 특정 단어의 개수
    result = math.log( (N) / (word_cnt[word] + 1) ) + 1
    return result

In [20]:
# TF-IDF -> TF의 값과 IDF 값을 곱한 수치 
X_tfidf = [ 
    [tf(w, doc) * idf(w) for w in vocab] for doc in tokens 
]
X_tfidf

[[0.0,
  0.0,
  0.0,
  0.3333333333333333,
  0.4684883693693881,
  0.4684883693693881,
  0.0,
  0.0],
 [0.3333333333333333,
  0.0,
  0.0,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.4684883693693881],
 [0.25,
  0.3513662770270411,
  0.3513662770270411,
  0.0,
  0.0,
  0.0,
  0.3513662770270411,
  0.0]]

In [21]:
pd.DataFrame(X_tfidf, columns = vocab)

,너무,배우의,연기가,영화가,재미있었다,정말,좋았다.,지루하다
0,0.000000,0.000000,0.000000,0.333333,0.468488,0.468488,0.000000,0.000000
1,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.468488
2,0.250000,0.351366,0.351366,0.000000,0.000000,0.000000,0.351366,0.000000


In [22]:
# TF-IDF는 scikit-learn에 class가 존재 
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vector = TfidfVectorizer(
    ngram_range= (1,1), 
    min_df=1
)
X =  vector.fit_transform(docs)
pd.DataFrame(
    X.toarray(), 
    columns = vector.get_feature_names_out()
)

,너무,배우의,연기가,영화가,재미있었다,정말,좋았다,지루하다
0,0.000000,0.000000,0.000000,0.473630,0.622766,0.622766,0.000000,0.000000
1,0.517856,0.000000,0.000000,0.517856,0.000000,0.000000,0.000000,0.680919
2,0.402040,0.528635,0.528635,0.000000,0.000000,0.000000,0.528635,0.000000


- 감정 분석에서 TF-IDF + SVM의 조합이 보편적으로 사용
    - 단어별로 feature가 생성이 되기 때문에 고차원 데이터가 생성이 되고 value가 0이 데이터가 많은 비율을 차지 하기 때문에 트리 구조를 중복으로 사용하는 배깅 부스팅은 어울리지 않는다
- ngram_range는 (1,2)로 사용을 하게 되면 '너무 좋았다', '정말 별로다' 이러한 단어 패턴도 포착 가능 
- 고차원으로 생성이 된 행렬 데이터를 차원 축소 기법을 이용하여 차원의 수를 줄이고 모델에 학습하여 성능을 평가하는 방법도 존재 

### N-gram
- 단어를 몇 개까지 하나로 볼것인가?
- 언어의 모델을 문장의 자연스러움을 수치화하는 통계적 방법 
- 수치화 값으로는 로그확률, 혼란도 
- 로그확률은 매우 크지 않은 음수인 경우 자연스러움 표현 
- 혼란도 낮을수록 자연스러움을 표현  

In [24]:
# 샘플 데이터를 생성 
texts = [
    '오늘 날씨가 좋다', 
    '오늘 기분이 좋다', 
    '내일 날씨가 맑다', 
    '기분이 아주 좋다', 
    '날씨가 아주 좋다'
]

In [25]:
from konlpy.tag import Okt

In [26]:
# 토큰화 함수 생성 
okt = Okt()
def tokenize(text):
    result = okt.morphs(text)
    return result

In [27]:
# 문장의 시작과 끝 부분에 태그를 추가 
tokens = [ ['<s>'] + tokenize(text) + ['</s>'] for text in texts ]
tokens

[['<s>', '오늘', '날씨', '가', '좋다', '</s>'],
 ['<s>', '오늘', '기분', '이', '좋다', '</s>'],
 ['<s>', '내일', '날씨', '가', '맑다', '</s>'],
 ['<s>', '기분', '이', '아주', '좋다', '</s>'],
 ['<s>', '날씨', '가', '아주', '좋다', '</s>']]

In [28]:
# unigram, bigram 카운트 확인 
from collections import Counter
unigram = Counter(w for token in tokens for w in token)
bigram = Counter(( token[i], token[i+1] ) for token in \
                 tokens for i in range(len(token) - 1) )
print(unigram)
print(bigram)

Counter({'<s>': 5, '</s>': 5, '좋다': 4, '날씨': 3, '가': 3, '오늘': 2, '기분': 2, '이': 2, '아주': 2, '내일': 1, '맑다': 1})
Counter({('좋다', '</s>'): 4, ('날씨', '가'): 3, ('<s>', '오늘'): 2, ('기분', '이'): 2, ('아주', '좋다'): 2, ('오늘', '날씨'): 1, ('가', '좋다'): 1, ('오늘', '기분'): 1, ('이', '좋다'): 1, ('<s>', '내일'): 1, ('내일', '날씨'): 1, ('가', '맑다'): 1, ('맑다', '</s>'): 1, ('<s>', '기분'): 1, ('이', '아주'): 1, ('<s>', '날씨'): 1, ('가', '아주'): 1})


In [29]:
# 위의 다중 for문을 풀어서 작성 
# for token in tokens:
#     for i in range(len(token) - 1):
#         print(token[i], token[i+1])

In [30]:
# 문서 내에서 단어의 개수 
V = len(unigram)
print(V)

11


In [31]:
# 확률이 0이 나오는 경우
# Add-1 Smoothing(Laplace Smoothing) 
# 분자에 + 1 -> 분모에는 + 문장에서의 단어의 개수  
def bigram_prob(prev, curr):
    # bigram[(prev, curr)] -> 연결된 두개의 단어의 개수
    # unigram[prev] -> 앞의 단어의 개수
    # 특정 단어 뒤에 단어가 나올 확률
    result = (bigram[(prev, curr)] + 1) / ( unigram[prev] + V)
    return result

# 로그 확률 -> 음수의 절대값이 크지 않다면 자연스러운 구조 
def log_prob(tokens):
    # tokens : 문장을 토큰화한 리스트
    # 문장의 시작과 끝에 태그 추가 
    seq = ['<s>'] + tokens + ['</s>']
    # 로그 확률의 누적합을 위해 초기 값 0.0 지정 
    log_p = 0.0
    for i in range(len(seq) - 1):
        # 확률 생성
        p = bigram_prob(seq[i], seq[i+1])
        # log_p에 누적합 
        log_p += math.log(p)
    return log_p


In [32]:
# 혼란도 
def perplexity(tokens):
    # tokens : 문장을 토큰화 한 리스트
    log_p = log_prob(tokens)
    T = len(tokens) + 1
    # 자연상수 계산식
    result = math.exp(-log_p / T)
    return result

In [33]:
# 학습된 unigram / bigram을 이용하여 확률을 계산하고 로그 확률, 
# 혼란도 확인 
new_texts = [
    '오늘 날씨가 맑다', 
    '기분이 내일 좋다', 
    '날씨가 아주 좋다'
]

result = []
for text in new_texts:
    tokens = tokenize(text)
    # 로그확률
    lp = log_prob(tokens)
    # 혼란도 
    perple = perplexity(tokens)
    result.append([text, lp, perple])


In [34]:
pd.DataFrame(result, columns = ['문장', '로그확률', '혼란도'])

,문장,로그확률,혼란도
0,오늘 날씨가 맑다,-8.536211,5.513735
1,기분이 내일 좋다,-9.694247,6.950749
2,날씨가 아주 좋다,-7.843064,4.799985


## 실습 문제 
- ratings_test.txt 파일을 로드 
- document 컬럼의 결측치를 제외하고 list 형태로 변환(데이터의 개수는 상위 5000개만)
- unigram, bigram 단어의 개수를 생성 
- 로그 확률, 혼란도 함수를 생성
- new_texts를 이용하여 로그확률, 혼란도를 생성 

In [35]:
new_texts = [
    '이 영화 좋네', 
    '시간이 가는줄 몰랐다', 
    '아 이런 내 시간', 
    '연기가 그닥 ... 배우 ... '
]

In [36]:
# 파일 로드 
test_df = pd.read_csv("../data/ratings_test.txt", sep='\t')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [37]:
# document의 결측치의 제외 / 상위의 5000개의 데이터를 사용
test_df.dropna(inplace=True)

In [38]:
# 리뷰의 중복을 제거 추가 되면 조금 더 확률이 정확
docs = test_df[:5000]['document'].to_list()

In [ ]:
# 토큰화 
# 형태소 생성
okt = Okt()

def tokenize(text):
    result = okt.morphs(text)
    return result

In [39]:
# 토큰화 된 데이터에 문장의 시작과 끝에 s태그를 추가 
tokens = [ ["<s>"] + tokenize(text) + ["</s>"] \
          for text in docs ]

In [ ]:
print(len(tokens))

In [ ]:
print(tokens[:3])

In [40]:
# unigram -> 문서 전체에서 단어 하나의 개수들
unigram = Counter(
    w for token in tokens for w in token
)

In [41]:
# bigram -> 문서 전체에서 (단어1, 단어2)의 개수들 
bigram = Counter(
    ( token[i], token[i+1] ) for token in tokens \
        for i in range(len(token) - 1)
)

In [42]:
print(len(unigram))
print(len(bigram))

13800
51067


In [43]:
# 문서 전체에서 사용이 되는 단어의 개수 
V = len(unigram)

In [53]:
# (단어1, 단어2) -> 확률 
def bigram_prob(prev, curr):
    result = ( bigram[ (prev, curr) ] + 1 ) /\
          ( unigram[prev] + V )
    return result

In [54]:
# 로그 확률 계산 하는 함수 
def log_prob(token):
    # token : 토큰화 된 단어들
    seq = ["<s>"] + token + ["</s>"]
    # 로그 확률 누적합이 들어갈 변수를 생성 
    log_p = 0.0
    for i in range(len(seq) - 1):
        p = bigram_prob( seq[i], seq[i+1] )
        log_p += math.log(p)
    return log_p

In [61]:
# 혼란도 함수 
def perplexity(token):
    # 로그 확률 함수를 호출 
    log_p = log_prob(token)
    T = len(token) + 1 # 마지막의 문장의 끝을 포함한 길이
    result = math.exp(-log_p / T)
    return result

In [62]:
new_texts

['이 영화 좋네', '시간이 가는줄 몰랐다', '아 이런 내 시간', '연기가 그닥 ... 배우 ... ']

In [44]:
result = []
for text in new_texts:
    tokens = tokenize(text)
    # 로그확률 계산 
    lp = log_prob(tokens)
    # 혼란도
    pp = perplexity(tokens)
    result.append( [text, lp, pp] )

In [45]:
pd.DataFrame(
    result, columns = ["문장", "로그 확률", '혼란도']
)

,문장,로그 확률,혼란도
0,이 영화 좋네,-28.977528,1400.216152
1,시간이 가는줄 몰랐다,-41.871799,4334.492438
2,아 이런 내 시간,-42.099771,4536.695383
3,연기가 그닥 ... 배우 ...,-52.497887,1807.496773


- 혼란도(perplexity) 
    - 매우 자연스럽다 -> 혼란도가 50 이하 
    - 매우 부자연스럽다 -> 혼란도가 500 이상

- 정리 
    - N-gram -> 이전 단어들을 보고 다음 단어의 확률을 예측
    - ADD-1 Smoothing -> 등장하지 않는 단어의 확률을 0이 아니라 0에 가까운 확률로 조정하는 방법 
    - 로그 확률 -> 확률의 누적합으로 절댓값이 클수록 부자연
    - 혼란도 -> 언어 모델이 문장을 얼마나 잘 예측하는가? 측정하는 지표